In [7]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import Parameter 



import numpy as np
import pandas as pd
import plotly.express as px

In [8]:
theta_param, phi_param = Parameter('theta_param'), Parameter('phi_param') 

qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.ry(theta_param, qreg_q[0])
circuit.rz(phi_param, qreg_q[0])

circuit.rx(-np.pi/2, qreg_q[0])
circuit.sdg(qreg_q[1])
circuit.h(qreg_q[1])
circuit.tdg(qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.t(qreg_q[1])
circuit.h(qreg_q[1])
circuit.s(qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[0])
circuit.rx(np.pi/2, qreg_q[0])
circuit.rx(np.pi/2, qreg_q[1])

circuit.rz(-phi_param, qreg_q[0])
circuit.ry(-theta_param, qreg_q[1])
circuit.measure(qreg_q[1], creg_c[1])

In [9]:
# IBMQ.save_account(token=)

N_shots=1024
provider = IBMQ.load_account()

#backend = least_busy(provider.backends(simulator=False))
#print(backend)
backend = provider.backends.ibmq_santiago

ibmqfactory.load_account:WARNING:2020-12-27 16:26:40,453: Credentials are already in use. The existing account in the session will be replaced.


In [10]:
num_pts = 1000
indices = np.arange(0, num_pts, dtype=float) + 0.5

theta = np.arccos(1 - 2*indices/num_pts)
phi = (np.pi * (1 + 5**0.5) * indices)%(2*np.pi)
coords=[]

for theta_elem,phi_elem in zip(theta,phi):
    coords.append((theta_elem,phi_elem))
    

In [11]:
runningjobs=[]
job_ids=[]
index=0

while index*75 < num_pts:
    if((index+1)*75<num_pts):
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:(index+1)*75]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_santiago.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    else:
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open('job_ids_santiago.txt', 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    index=index+1

Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f7762302550>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f77612f4050>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f7761b8cdd0>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f77622a3890>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f77621ee090>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f7761c16910>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f776244b890>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 